In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
#sys.path.append("~/catkin_ws/src/ros-eurobot-2018/eurobot_decision_maker/src/python/executor")
# from executor import *

In [2]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])
## HERE HERE HERE

In [3]:
class StrategyOptimizer:
    """
        Single (big) robot optimizer
    """
    actions = {
        "heaps": 6,
        "funny": 2,
        "disposal": 1,
        "base": 1
    }
    
    action_places = {
        "heaps": np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]], dtype=np.float64),
        "funny": np.array([[10, 113, 0], [190, 10, 0]], dtype=np.float64),
        "disposal": np.array([[10, 61, 0]],dtype=np.float64),
        "base": np.array([[15, 15, 0]],dtype=np.float64)
    }
    action_times = {
        "heaps" : [17,17,17,17,17,17],
        "base" : [0],
        "disposal" : [5],
        "funny" : [3,3]
    }
    def __init__(self, **kvargs):
        self.model = grb.Model("eurobot")
        self.actions = {}
        
        # choose some certain actions
        # by default all from Optimizer.actions
        if "actions" in kvargs:
            try:
                for a in kvargs["actions"]:
                    self.actions[a] = StrategyOptimizer.actions[a]
            except:
                print("Error: wrong actions in kvargs")
        else:
            self.actions.update(StrategyOptimizer.actions)
        
        
        # N for total available actions (from them we can choose)
        self.N = sum(self.actions.values())
        # S for total maximal number of actions can be created by optimizer
        self.S = self.N 
        self.x = self.model.addVars(range(self.S), range(self.N), name = 'x', vtype=grb.GRB.BINARY)
        
        #define positions of all actions in x variable
        self.actions_iters = {}
        i = 0
        for a in self.actions:
            self.actions_iters[a] = list(range(i,i+self.actions[a]))
            i += self.actions[a]
        print(self.actions_iters)
        self.extra_conditions = []
       
        # set default Time Matrix
        self.create_default_time_matrix()
        
        self.action_times = copy.deepcopy(StrategyOptimizer.action_times)
        
        # create time constraint/objective
        self.create_time_constraint()
        
        # create points contraint/objective
        self.create_points_constraint()
        
        self.Time = 100.0
        self.Points = 220.0
    
        self.objective = "time"
        if "objective" in kvargs:
            if kvargs["objective"] in ["time","points"]:
                self.objective = kvargs["objective"]
        
        self.banned_heaps = None
        
    def reset_constraints(self):
        self.model.remove(self.model.getConstrs())
        
    def create_default_time_matrix(self):
        ## Calculate time matrix
        # places with cubes
        #heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])

        # starting place of robot
        # to be redefined at the start of the algorithm
        #base_point  = np.array([[15, 15, 0]])

        # disposal place 
        # TODO: add another side of field
        #disposal_point = np.array([[10, 61, 0]])

        # funny action places
        #funny_points = np.array([[10, 113, 0], [190, 10, 0]])

        action_places = np.concatenate(tuple(StrategyOptimizer.action_places[action] for action in self.actions ))

        # time is simple distance / v
        v = 30

        self.time_travel = np.sum((action_places[np.newaxis,:]-action_places[:,np.newaxis]) ** 2, axis = 2)**0.5 / v
        
    def add_unique_action_and_limited_number_actions_constraints(self):
        
        self.model.addConstrs((self.x.sum(s, '*') <= 1 for s in iterer(self.S)));
        self.model.addConstrs((self.x.sum(s, '*') - self.x.sum(s+1,'*') >= 0 for s in range(self.S-1)))
        
    def add_default_constraints(self):
        # unique action per 's'-tep
        # self.model.addConstrs((self.x.sum(s, '*') == 1 for s in iterer(self.S)));
        
        self.add_unique_action_and_limited_number_actions_constraints();
        
        # each action can be done only once
        self.model.addConstrs((self.x.sum('*', i) <= 1 for i in range(self.N)));
            
        # now max 3 heaps
        self.model.addConstr(grb.quicksum((self.x.sum('*',i) for i in self.actions_iters['heaps'])) <= 3 );
        
        # disposal after all 3 heaps picking
        for s, k in iterer(self.S,self.actions["heaps"]):
            self.model.addConstr( grb.quicksum( (self.x[s_,self.actions_iters["disposal"][0]] for s_ in range(s,self.S)) )
                                 >= self.x[s,k] );
        # starting from base point
        self.model.addConstr(self.x[0,self.actions_iters["base"][0]] == 1);
    def add_banned_heaps(self, bans=None): # [False, True, False, False, False, False] to ban second heap
        if bans == None:
            return
        else:
            self.banned_heaps = bans
    def set_banned_heaps_constraint(self):
        if self.banned_heaps == None:
            return
        else:
            for i, k in enumerate(self.actions_iters["heaps"]):
                if self.banned_heaps[i]:
                    self.model.addConstr(self.x.sum('*',k) == 0)
    def add_action_times(self, action_times):
        for a in self.actions.keys():
            self.action_times[a] = action_times[a]
        print(self.action_times)
    def create_time_constraint(self):
        time_travel_constr_gen = (self.x[s, k_1] * self.x[s+1, k_2] * self.time_travel[k_1, k_2] for  s,k_1, k_2 in 
                                  #[(0,9, 2)])
                                  iterer(self.S-1,self.N,self.N))
        left_part_time_1 = grb.quicksum(time_travel_constr_gen)
        left_part_time_2 = 0
        for a in self.actions.keys():
            print(self.action_times[a])
            print(self.actions_iters[a])
            left_part_time_2 += grb.quicksum(self.x.sum('*',k)*self.action_times[a][i] for i,k in enumerate(self.actions_iters[a]) )
        
        self.left_part_time = left_part_time_1 + left_part_time_2
    
    def create_points_constraint(self):
        #TODO: rewrite with points array
        self.total_points = 45*grb.quicksum((self.x.sum('*',k) for k in self.actions_iters["heaps"]))  + \
            30*self.x.sum('*',self.actions_iters["funny"][0]) + 55*self.x.sum('*',self.actions_iters["funny"][1]);
    def set_objective(self):
        """
            minimize "time"
            or
            maximize "points"
        """
        if self.objective=="time":
            self.model.addConstr(self.total_points == self.Points, 'points')
            self.model.setObjective(self.left_part_time, grb.GRB.MINIMIZE)
        elif self.objective=="points":
            self.model.addConstr(self.left_part_time <= self.Time, 'time')
            self.model.setObjective(self.total_points, grb.GRB.MAXIMIZE)
    
    def set_done_actions_constraints(self, done_actions):
        for i, a_index in enumerate(done_actions):
            a,index = a_index
            self.model.addConstr(self.x[i,self.actions_iters[a][index]] == 1)
    
    def prepare_model(self, banned_heaps=None, done_actions=None):
        self.reset_constraints()
        self.add_default_constraints()
        self.set_objective()
        if banned_heaps != None:
            self.add_banned_heaps(banned_heaps)
        self.set_banned_heaps_constraint()
        
        if done_actions != None:
            self.set_done_actions_constraints(done_actions)
        
    def run(self):
        self.model.update()
        self.model.optimize()
        self.model.printAttr('x')

    def get_solution(self, **kvargs):
        view = "numbers"
        if "view" in kvargs and kvargs['view'] in ["numbers", "text"]:
            view = kvargs["view"]
        
        result = []
        
        for a in self.x.items():
            if abs(a[1].X - 1) < 0.1:
                result.append(a[0][1])

        if view == "numbers":
            return result
        else:
            result_tuples = []
            for a in result:
                for name, indices in self.actions_iters.items():
                    if a in indices:
                        result_tuples.append((name,a-indices[0]))
            return result_tuples

In [4]:
class BothStrategyOptimizer:
    actions = {
        "heaps": 6,
        "funny": 2,
        "disposalbig": 1,
        "disposalsmall": 1,
        "basebig": 1,
        "basesmall": 1,
        "collectdirty": 1,
        "collectclean": 1,
        "unloaddirty" : 1
    }
    action_places = {
        "heaps": np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]], dtype=np.float64),
        "funny": np.array([[10, 113, 0], [190, 10, 0]], dtype=np.float64),
        "disposalbig": np.array([[10, 51, 0]],dtype=np.float64),
        "disposalsmall": np.array([[10, 71, 0]],dtype=np.float64),
        "basebig": np.array([[20, 15, 0]],dtype=np.float64),
        "basesmall": np.array([[55, 15, 0]],dtype=np.float64),
        "collectdirty": np.array([[239, 180, 0]],dtype=np.float64),
        "collectclean": np.array([[20, 84, 0]],dtype=np.float64),
        "unloaddirty": np.array([[120, 160, 0]],dtype=np.float64)
    }
    action_times = {
        "heaps" : [17,17,17,17,17,17],
        "funny" : [3,3],
        "disposalbig": [5],
        "disposalsmall": [2],
        "basebig": [0],
        "basesmall": [0],
        "collectdirty": [15],
        "collectclean": [15],
        "unloaddirty" : [5]
    }
    action_points = {
        "heaps": [45, 45, 45, 45, 45, 45],
        "funny": [30, 50],
        "disposalbig": [0],
        "disposalsmall": [0],
        "basebig": [0],
        "basesmall": [0],
        "collectdirty": [10],
        "collectclean": [10+60],
        "unloaddirty" : [20]
    }
    def __init__(self, **kvargs):
        self.model = grb.Model("eurobot")
        # self.actions = {}
        
        # choose some certain actions
        # by default all from Optimizer.actions
#         if "actions" in kvargs:
#             try:
#                 for a in kvargs["actions"]:
#                     self.actions[a] = StrategyOptimizer.actions[a]
#             except:
#                 print("Error: wrong actions in kvargs")
#         else:
#             self.actions.update(StrategyOptimizer.actions)
        
        
        # N for total available actions (from them we can choose)
        self.N = sum(self.actions.values())
        # S for total maximal number of actions can be created by optimizer
        self.S = self.N 
        
        # x for BIG robot, y for SMALL robot
        self.x = self.model.addVars(range(self.S), range(self.N), name = 'x', vtype=grb.GRB.BINARY)
        self.y = self.model.addVars(range(self.S), range(self.N), name = 'y', vtype=grb.GRB.BINARY)
        
        #define positions of all actions in x,y variables
        self.actions_iters = {}
        i = 0
        for a in self.actions:
            self.actions_iters[a] = list(range(i,i+self.actions[a]))
            i += self.actions[a]
        print(self.actions_iters)
        self.extra_conditions = []
       
        
        self.action_times = copy.deepcopy(BothStrategyOptimizer.action_times)
        
        
        # set default Time Matrix
        self.create_default_time_matrix()
        
        # create time constraint/objective
        self.total_points = self.create_points_constraint()
        
        # create points contraint/objective
        self.left_part_time_x = self.create_time_constraint(self.x)
        self.left_part_time_y = self.create_time_constraint(self.y)
        
        self.Time = 100.0
        self.Points = 45*4 + 30 + 50 + 100
    
        self.objective = "mixed"
        if "objective" in kvargs:
            if kvargs["objective"] in ["time","points"]:
                self.objective = kvargs["objective"]
        
        self.banned_heaps = None
    
    def reset_constraints(self):
        self.model.remove(self.model.getConstrs())
        
    def create_default_time_matrix(self):
        ## Calculate time matrix
        # places with cubes
        #heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])

        # starting place of robot
        # to be redefined at the start of the algorithm
        #base_point  = np.array([[15, 15, 0]])

        # disposal place 
        # TODO: add another side of field
        #disposal_point = np.array([[10, 61, 0]])

        # funny action places
        #funny_points = np.array([[10, 113, 0], [190, 10, 0]])

        action_places = np.concatenate(tuple(BothStrategyOptimizer.action_places[action] for action in self.actions ))

        # time is simple distance / v
        v = 30

        self.time_travel = np.sum((action_places[np.newaxis,:]-action_places[:,np.newaxis]) ** 2, axis = 2)**0.5 / v
        
    def add_unique_action_and_limited_number_actions_constraints(self):
        # unique action per 's'-tep
        # self.model.addConstrs((self.x.sum(s, '*') == 1 for s in iterer(self.S)));
        
        self.model.addConstrs((self.x.sum(s, '*') <= 1 for s in iterer(self.S)));
        self.model.addConstrs((self.x.sum(s, '*') - self.x.sum(s+1,'*') >= 0 for s in range(self.S-1)));
        
        self.model.addConstrs((self.y.sum(s, '*') <= 1 for s in iterer(self.S)));
        self.model.addConstrs((self.y.sum(s, '*') - self.y.sum(s+1,'*') >= 0 for s in range(self.S-1)));
    
    def add_action_only_once_constraints(self):
        # each action can be done only once
        self.model.addConstrs((self.x.sum('*', i) <= 1 for i in range(self.N)));
        self.model.addConstrs((self.y.sum('*', i) <= 1 for i in range(self.N)));
        
    def add_maximum_amount_of_heaps_constraint(self, var, num):
        # now max 3 heaps for BIG, 1 for SMALL
        self.model.addConstr(grb.quicksum((var.sum('*',i) for i in self.actions_iters['heaps'])) <= num );
        
    def add_one_action_after_another_constraints(self, var, action_before, action_after):
        if action_before in self.actions_iters and action_after in self.actions_iters:
            for s, before, after in iterer(self.S, self.actions_iters[action_before], self.actions_iters[action_after]):
                self.model.addConstr( grb.quicksum( (var[s_, after] for s_ in range(s, self.S))) >= var[s,before]);
            
    def add_fixed_action_constraint(self, var, action_number, order):
        self.model.addConstr(var[order,action_number] == 1)
        
    def add_robot_cant_do_action_constraint(self, var, action_name):
        print(self.actions_iters[action_name])
        self.model.addConstrs((var.sum('*',k) == 0 for k in self.actions_iters[action_name]))
    def add_unique_for_all_robots_action_constraint(self, action_name):
        self.model.addConstrs((self.x.sum('*', k) + self.y.sum('*', k) <= 1 for k in self.actions_iters[action_name] ))
    def add_default_constraints(self):
        
        self.add_unique_action_and_limited_number_actions_constraints();
        
        self.add_action_only_once_constraints();
        
        self.add_maximum_amount_of_heaps_constraint(self.x, 3);
        self.add_maximum_amount_of_heaps_constraint(self.y, 1);
        
        self.add_one_action_after_another_constraints(self.x, "heaps", "disposalbig");
        self.add_one_action_after_another_constraints(self.y, "heaps", "disposalsmall");
        
        self.add_one_action_after_another_constraints(self.y, "collectdirty", "unloaddirty");
        self.add_one_action_after_another_constraints(self.y, "collectclean","collectdirty");
        # disposal after all 3 heaps picking
        # for s, k in iterer(self.S,self.actions["heaps"]):
        #     self.model.addConstr( grb.quicksum( (self.x[s_,self.actions_iters["disposal"][0]] for s_ in range(s,self.S)) )
        #                          >= self.x[s,k] );
        
        self.add_robot_cant_do_action_constraint(self.x, "collectdirty");
        self.add_robot_cant_do_action_constraint(self.x, "collectclean");
        self.add_robot_cant_do_action_constraint(self.x, "unloaddirty");
        self.add_robot_cant_do_action_constraint(self.x, "disposalsmall");
        
        self.add_robot_cant_do_action_constraint(self.y, "disposalbig");
        
        self.add_unique_for_all_robots_action_constraint("funny");
        # starting from base point
        self.add_fixed_action_constraint(self.x, self.actions_iters["basebig"][0], 0);
        self.add_fixed_action_constraint(self.y, self.actions_iters["basesmall"][0], 0);
        
    def add_banned_heaps(self, bans=None): # [False, True, False, False, False, False] to ban second heap
        if bans == None:
            return
        else:
            self.banned_heaps = bans
            
    def set_banned_heaps_constraint(self):
        if self.banned_heaps == None:
            return
        else:
            for i, k in enumerate(self.actions_iters["heaps"]):
                if self.banned_heaps[i]:
                    self.model.addConstr(self.x.sum('*',k) == 0)
                    self.model.addConstr(self.y.sum('*',k) == 0)
                    
    def add_action_times(self, action_times):
        for a in self.actions.keys():
            self.action_times[a] = action_times[a]
        print(self.action_times)
    def create_time_constraint(self, var):
        time_travel_constr_gen = (var[s, k_1] * var[s+1, k_2] * self.time_travel[k_1, k_2] for  s,k_1, k_2 in 
                                  #[(0,9, 2)])
                                  iterer(self.S-1,self.N,self.N))
        left_part_time_1 = grb.quicksum(time_travel_constr_gen)
        left_part_time_2 = 0
        for a in self.actions.keys():
            print(self.action_times[a])
            print(self.actions_iters[a])
            left_part_time_2 += grb.quicksum(var.sum('*',k)*self.action_times[a][i] for i,k in enumerate(self.actions_iters[a]) )
        
        return left_part_time_1 + left_part_time_2
    
    def create_points_constraint(self):
        #TODO: rewrite with points array
        total_points = 0
        
        # heaps picking
        total_points += self.x.sum('*',self.actions_iters['disposalbig'][0]) * \
            grb.quicksum((score * self.x.sum('*',k) for score, k in zip(self.action_points["heaps"],self.actions_iters["heaps"])))
        total_points += self.y.sum('*',self.actions_iters['disposalsmall'][0]) * \
            grb.quicksum((score * self.y.sum('*',k) for score, k in zip(self.action_points["heaps"],self.actions_iters["heaps"])))
        
        # other actions
        for a in self.actions.keys() - "heaps":
            total_points += grb.quicksum((score*(self.x.sum('*',k) + self.y.sum('*',k)) for score, k in zip(self.action_points[a],self.actions_iters[a])))
        
        
        # total_points = 45*grb.quicksum((self.x.sum('*',k) for k in self.actions_iters["heaps"]))  + \
        #    30*self.x.sum('*',self.actions_iters["funny"][0]) + 55*self.x.sum('*',self.actions_iters["funny"][1]);
        
        return total_points
    
    def set_objective(self):
        """
            minimize "time"
            or
            maximize "points"
            or
            ~both
        """
        if self.objective=="time":
            self.model.addConstr(self.total_points == self.Points, 'points')
            self.model.setObjective(self.left_part_time_x/2 + self.left_part_time_y/2, grb.GRB.MINIMIZE)
        elif self.objective=="points":
            self.model.addConstr(self.left_part_time_x <= self.Time, 'time_big')
            self.model.addConstr(self.left_part_time_y <= self.Time, 'time_small')
            self.model.setObjective(self.total_points, grb.GRB.MAXIMIZE)
        elif self.objective=="mixed":
            self.model.addConstr(self.left_part_time_x <= self.Time, 'time_big')
            self.model.addConstr(self.left_part_time_y <= self.Time, 'time_small')
            self.model.setObjective(self.total_points - self.left_part_time_x/100 - self.left_part_time_y/100, grb.GRB.MAXIMIZE)
    
    def set_done_actions_constraints(self, var, done_actions):
        for i, a_index in enumerate(done_actions):
            a,index = a_index
            self.add_fixed_action_constraint(var, a, index)
    
    def prepare_model(self, banned_heaps=None, done_actions=None):
        self.reset_constraints()
        self.add_default_constraints()
        self.set_objective()
        if banned_heaps != None:
            self.add_banned_heaps(banned_heaps)
        self.set_banned_heaps_constraint()
        
        if done_actions != None:
            if "big" in done_actions:
                self.set_done_actions_constraints(self.x, done_actions['big'])
            if "small" in done_actions:
                self.set_done_actions_constraints(self.y, done_actions['small'])
        
    def run(self):
        self.model.update()
        self.model.optimize()
        self.model.printAttr('x')

    def get_solution(self, **kvargs):
        view = "numbers"
        if "view" in kvargs and kvargs['view'] in ["numbers", "text"]:
            view = kvargs["view"]
        
        result = [[],[]]
        
        for a in self.x.items():
            if abs(a[1].X - 1) < 0.1:
                result[0].append(a[0][1])
        
        for a in self.y.items():
            if abs(a[1].X - 1) < 0.1:
                result[1].append(a[0][1])

        if view == "numbers":
            return result
        else:
            result_tuples = [[],[]]
            for i in range(2):
                for a in result[i]:
                    for name, indices in self.actions_iters.items():
                        if a in indices:
                            result_tuples[i].append((name,a-indices[0]))
            return result_tuples
        

In [8]:
opt = StrategyOptimizer(objective='time')
# opt.add_banned_heaps([False,True,True,False,False,False])
opt.prepare_model(None,None)
opt.run()
print(opt.get_solution(view="text"))

{'heaps': [0, 1, 2, 3, 4, 5], 'funny': [6, 7], 'disposal': [8], 'base': [9]}
[17, 17, 17, 17, 17, 17]
[0, 1, 2, 3, 4, 5]
[3, 3]
[6, 7]
[5]
[8]
[0]
[9]
Optimize a model with 92 rows, 100 columns and 911 nonzeros
Model has 810 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [3e+00, 2e+01]
  QObjective range [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 17 rows and 25 columns
Presolve time: 0.00s
Presolved: 603 rows, 603 columns, 2274 nonzeros
Variable types: 0 continuous, 603 integer (603 binary)

Root relaxation: objective 6.057790e+01, 120 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   60.57790    0   54          -   60.57790      -     -    0s
H    0     0                     100.4

In [45]:

class CubePickingOptimizer:
    def __init__(self):
        self.Allowed_seq_1 = []
        self.Allowed_seq_2 = []
        self.Allowed_seq_3 = []

    def set_plan(self, plan):
        self.allowed_cubes_seq(plan)

    def allowed_cubes_seq(self, plan):
        all_cubes = list(range(5))
        oth = list(set(all_cubes) - set(plan))
        sds = [plan[0], plan[2]]
        cnt = plan[1]

        self.Allowed_seq_1 = [
            (oth[0],),
            (oth[1],),
            (sds[0],),
            (sds[1],)
        ]

        self.Allowed_seq_2 = [
            (oth[0], sds[0]),
            (oth[1], sds[0]),
            (oth[0], sds[1]),
            (oth[1], sds[1]),
            (oth[0], oth[1]),
            (oth[1], oth[0]),
            (sds[1], cnt),
            (sds[0], cnt)
        ]

        self.Allowed_seq_3 = [
            (oth[0], sds[0], cnt),
            (oth[1], sds[0], cnt),
            (oth[0], sds[1], cnt),
            (oth[1], sds[1], cnt),
            (oth[0], oth[1], sds[0]),
            (oth[0], oth[1], sds[1]),
            (oth[1], oth[0], sds[0]),
            (oth[1], oth[0], sds[1]),
            (cnt, sds[0], oth[0]),
            (cnt, sds[0], oth[1]),
            (cnt, sds[1], oth[0]),
            (cnt, sds[1], oth[1]),
            (sds[0], oth[0], oth[1]),
            (sds[1], oth[0], oth[1]),
            (sds[0], oth[1], oth[0]),
            (sds[1], oth[1], oth[0]),
            (sds[1], cnt, sds[0]),
            (sds[0], cnt, sds[1])
        ]

    def find_optimal_sequence(self, key):
        # dynamic searching of optimal piking sequence
        seqs = [{((), (), ()): []}, {}, {}, {}]
        for i in range(3):
            for last_picked_cubes, last_seq in seqs[i].items():
                for new_picked_cubes, new_seq in self.pick_one_heap(last_picked_cubes, last_seq):
                    if self.check_safety(new_picked_cubes, i + 1):
                        old_value = seqs[i + 1].get(new_picked_cubes)
                        if old_value is None:
                            seqs[i + 1][new_picked_cubes] = new_seq
                        else:
                            seqs[i + 1][new_picked_cubes] = min(new_seq, seqs[i + 1][new_picked_cubes], key=key)
        return min(seqs[3].values(), key=key)

    def pick_one_heap(self, picked_cubes, seq):
        remain_cubes = [0, 1, 2, 3]
        yield from self.pick_one_cube(picked_cubes, seq, [3, 4, 1], remain_cubes)
        yield from self.pick_one_cube(picked_cubes, seq, [2, 1, 4, 1], remain_cubes)
        yield from self.pick_one_cube(picked_cubes, seq, [2, 2, 4], remain_cubes)
        yield from self.pick_one_cube(picked_cubes, seq, [1, 2, 4, 1], remain_cubes)
        yield from self.pick_one_cube(picked_cubes, seq, [1, 1, 1, 4, 1], remain_cubes)

    def check_safety(self, towers_cubes, n):
        n_tr = 0
        for tower in towers_cubes:
            if len(tower) > 3 and tower[-3:] in self.Allowed_seq_3[8:]:
                n_tr += 1
        return n_tr >= n

    def check_tower(self, tower_cubes):
        n = len(tower_cubes)
        if n == 1:
            return tower_cubes in self.Allowed_seq_1
        elif n == 2:
            return tower_cubes in self.Allowed_seq_2
        elif n < 6:
            return tower_cubes[-3:] in self.Allowed_seq_3
        else:
            return False

    @staticmethod
    def pick(remain_cubes, n_cubes):
        if n_cubes == 4:
            for i in range(3):
                picked_cubes = [[], [], []]
                picked_cubes[i] = [4]
                yield remain_cubes, picked_cubes, [i]
        elif n_cubes == 1:
            for i in range(len(remain_cubes)):
                for j in range(3):
                    picked_cubes = [[], [], []]
                    picked_cubes[j] = [remain_cubes[i]]
                    yield remain_cubes[:i] + remain_cubes[i + 1:], picked_cubes, [j]
        elif n_cubes == 2:
            for i in range(4):
                picked = [i, (i + 1) % 4]
                if all([(x in remain_cubes) for x in picked]):
                    yield list(set(remain_cubes) - set(picked)), [[picked[0]], [picked[1]], []], [0, 1]
                    yield list(set(remain_cubes) - set(picked)), [[], [picked[0]], [picked[1]]], [1, 2]
            for i in range(4):
                picked = [i, (i + 2) % 4]
                if all([(x in remain_cubes) for x in picked]):
                    yield list(set(remain_cubes) - set(picked)), [[picked[0]], [], [picked[1]]], [0, 2]
        elif n_cubes == 3:
            yield [3], [[0], [1], [2]], [0, 1, 2]
            yield [0], [[1], [2], [3]], [0, 1, 2]
            yield [1], [[2], [3], [0]], [0, 1, 2]
            yield [2], [[3], [0], [1]], [0, 1, 2]

    def pick_one_cube(self, picked_cubes, seq, strat, remain_cubes):
        if strat:
            for new_remain_cubes, new_picked_cubes, towers in self.pick(remain_cubes, strat[0]):
                new_towers_cubes = tuple([picked_cubes[i] + tuple(new_picked_cubes[i]) for i in range(3)])
                if all([self.check_tower(new_towers_cubes[x]) for x in towers]):
                    yield from self.pick_one_cube(new_towers_cubes, seq + [new_picked_cubes], strat[1:],
                                                  new_remain_cubes)
        else:
            yield picked_cubes, seq

    @staticmethod
    def picking_places_and_states(seq):
        places = []
        states = []
        for i, pick in enumerate(seq):
            n = sum(map(lambda x: len(x), pick))
            place = 0
            state = 0
            for j, cube in enumerate(pick):
                if cube:
                    if cube[0] == 4:
                        state = 1
                        place = places[-1]
                    elif len(states) != 0 and states[-1] == 1 and n == 1:
                        place = places[-1] = (cube[0] - j - 1) % 4
                        state = 2
                    else:
                        place = (cube[0] - j + 1) % 4
                        state = 0
                    break
            places.append(place)
            states.append(state)
        return places, states

    @staticmethod
    def dif_point(x1, x2, n):
        return min((x1 - x2) % n, (x1 - x2) % n)

    def rotation_time(self, places, states, start_points=None):
        if start_points is not None:
            time = self.dif_point(start_points[0], places[0], 4)
            n_start = 1
        else:
            time = 0
            n_start = 0
        for i in range(len(places)):
            if i != 0 and not (states[i] == 0 and states[i - 1] != 0):
                time += min((places[i] - places[i - 1]) % 4, (places[i - 1] - places[i]) % 4)
            elif i != 0 and states[i] == 0 and states[i - 1] != 0 and start_points is not None:
                time += self.dif_point(start_points[n_start], places[i], 4)
                n_start += 1
        return time

    @staticmethod
    def move_time(places, states):
        time = 0
        for i in range(len(places)):
            if i != 0 and (states[i] == 1 and states[i - 1] == 0 or states[i] == 2 and states[i - 1] == 1):
                time += 1
        return time

    def get_fun_time(self, k_r=1, k_m=1, k_p=1, start_points=None):
        def time(seq):
            places, states = self.picking_places_and_states(seq)
            t_r = self.rotation_time(places, states, start_points)
            t_m = self.move_time(places, states)
            t_p = len(seq)
            return t_r * k_r + t_m * k_m + t_p * k_p
        return time


In [46]:
optimizer = CubePickingOptimizer()
optimizer.set_plan([2, 4, 3])
time = optimizer.get_fun_time(k_r=1, k_m=1.5, k_p=2)
seq = optimizer.find_optimal_sequence(time)
for s in seq:
    print(s)
print(time(seq))

[[0], [1], []]
[[3], [], []]
[[4], [], []]
[[2], [], []]
[[], [3], [0]]
[[], [], [1]]
[[], [4], []]
[[], [2], []]
[[0], [1], [2]]
[[], [], [4]]
[[], [], [3]]
35.0


In [62]:
a = {'a':1,'b':1}
for i in a.keys() - 'a':
    print(i)

b
